<a href="https://colab.research.google.com/github/KSaiBharadwaj/fraudanalytics/blob/main/DSO562_HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries

In [2]:
import pandas as pd
from datetime import date,datetime,time

Problem 1: Computing the day of the week of 07-04-1776.

In [3]:
idd = date.fromisoformat("1776-07-04")
iddt = datetime.fromisoformat("1776-07-04")

print("Using date - day of the week is "+idd.strftime("%A"))
print("Using datetime - day of the week is "+iddt.strftime("%A"))

Using date - day of the week is Thursday
Using datetime - day of the week is Thursday


Problem 2: Number of days between World War 1 (1919-11-11) and World War 2 (1939-09-01)

In [4]:
ww1 = datetime.fromisoformat("1919-11-11")
ww2 = datetime.fromisoformat("1939-09-01")

print(ww2 - ww1)

7234 days, 0:00:00


Problem 3: RFM Analysis of Small mail order Catalog company

In [6]:
import numpy as np
from sklearn import preprocessing

dataRfm = pd.read_csv("/content/sample_data/Date RFMdata.csv")
dataRfm.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3915 entries, 0 to 3914
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Customer  3915 non-null   int64  
 1   Date      3915 non-null   object 
 2   Size      3915 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 91.9+ KB


In [9]:
# Date Column is stored as object, converting into datetime format
dataRfm["CDate"] = pd.to_datetime(dataRfm["Date"])
dataRfm.info()
today = datetime.fromisoformat("2023-09-08")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3915 entries, 0 to 3914
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Customer  3915 non-null   int64         
 1   Date      3915 non-null   object        
 2   Size      3915 non-null   float64       
 3   CDate     3915 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 122.5+ KB


In [8]:
dataRfm.head()

,Customer,Date,Size,CDate
0,22,9/15/2017,100.0,2017-09-15
1,58,7/25/2021,100.0,2021-07-25
2,142,1/4/2013,100.0,2013-01-04
3,160,1/25/2021,100.0,2021-01-25
4,203,5/16/2022,100.0,2022-05-16


In [17]:
# Calculating Recency
recency = dataRfm.groupby('Customer').agg({"CDate" : lambda x : ((today-x.max()).days)})

In [31]:
# Calculating Frequency and Monetary
frequency = dataRfm.groupby('Customer')["Size"].count()
monetary = dataRfm.groupby('Customer')["Size"].sum()

# Creating Recency Table

recency.columns = ["Recency"]
frequency.columns = ["Frequency"]
monetary.columns = ["Monetary"]

RFM = pd.concat([recency,frequency,monetary], axis=1)

# Scaling RFM to a scale 0f 0 - 10

rfmScaled = preprocessing.minmax_scale(RFM)

rfmScaled[:,0] = (1 - rfmScaled[:,0])*10
rfmScaled[:,1] = (rfmScaled[:,1])*10
rfmScaled[:,2] = (rfmScaled[:,2])*10

# Cocainating the Columns to the Main table

RFM["Scaled_Recency"] = rfmScaled[:,0]
RFM["Scaled_Frequency"] = rfmScaled[:,1]
RFM["Scaled_Monetary"] = rfmScaled[:,2]

# Calulating the Sum for RFM Analysis

RFM["SUM_Scaled_RFM"] = RFM["Scaled_Recency"] + RFM["Scaled_Frequency"] + RFM["Scaled_Monetary"]

RFM["SUM_Scaled_RFM"].sort_values(ascending=False).head(5)

Customer
180    28.406910
37     26.915562
161    25.660998
224    23.777619
114    22.881434
Name: SUM_Scaled_RFM, dtype: float64

Plotting using Plotly

In [36]:
import plotly.express as px
import plotly.io as pio

figure = px.scatter_3d(
    data_frame=RFM,
    x = "Scaled_Recency",
    y = "Scaled_Frequency",
    z = "Scaled_Monetary",
    template = "plotly_dark",
    title = "RFM Plot"
)

pio.show(figure)